# Demo for paper "First Order Motion Model for Image Animation"
Modified by Ting Yew, Davis

YouTube guide [here](https://youtu.be/zZr3EHLBm4g).

Crop, resize, and cut videos: - https://ezgif.com/

**Clone repository**

In [ ]:
# !git clone https://github.com/AliaksandrSiarohin/first-order-model

**Load driving video and source image**

In [107]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
import os

from pathlib import Path
path = Path('C:/Users/kting/Documents/GitHub/DeepFake/first-order-motion-model/')

In [150]:
def display2(driving, pic_ls, pred_ls=None):
    """
    Creates 2 rows:
    1st row is Driving video + still source images
    2nd row is Driving video + predictions
    """
    
    fig = plt.figure(figsize=(12 , 6))

    im_ls = []
    for i in range(len(driving)):
        
        # 1st Row
        frame = [driving[i]] #i-th frame of driving video
        for p in pic_ls: frame.append(p) # append all still images in pic_ls
        frame = np.concatenate(frame, axis=1) # axis-1 concatenation places them all in a row
            
        # 2nd Row
        if pred_ls is not None:
            row2 = [driving[i]]
            for p in pred_ls: row2.append(p[i])
            row2 = np.concatenate(row2, axis=1)
            row1 = [frame]
            row1.append(row2)
            frame = np.concatenate(row1, axis=0)
                
        im = plt.imshow(frame, animated=True)

        plt.axis('off')
        im_ls.append([im])

    ani = animation.ArtistAnimation(fig, im_ls, interval=37, repeat_delay=1000)
    plt.close()
    return ani

In [151]:
arr = os.listdir(path/'Pictures')
arr

['me_art.jpg', 'me_home.jpg', 'me_la.jpg', 'me_museum.jpg', 'me_red.jpg']

In [152]:
N = 256 # Must be 256, because model is trained on size 256x256
imgs = []
for name in os.listdir(path/'Pictures'):
    img = imageio.imread(path/'Pictures'/name)
    print(f'Resized {name}.')
    imgs.append(resize(img, (N, N))[..., :3])

Resized me_art.jpg.
Resized me_home.jpg.
Resized me_la.jpg.
Resized me_museum.jpg.
Resized me_red.jpg.


In [153]:
driving_video = imageio.mimread(path/'trumptalk.mp4')

N = 256 
driving = [resize(frame, (N, N))[..., :3] for frame in driving_video]

In [154]:
HTML(display2(driving, imgs).to_html5_video())

**Create a model and load checkpoints**

In [155]:
import os
os.chdir("C:\\Users\\kting\\Documents\\GitHub\\DeepFake\\first-order-model")

In [156]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=path/'vox-cpk.pth.tar')

**Perform image animation**

In [157]:
from demo import make_animation
from skimage import img_as_ubyte

In [158]:
preds = [make_animation(p, driving, generator, kp_detector, relative=True) for p in imgs]

100%|████████████████████████████████████████████████████████████████████████████████| 211/211 [00:13<00:00, 15.97it/s]


In [159]:
HTML(display2(driving, imgs, preds).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [160]:
preds = [make_animation(p, driving, generator, kp_detector, relative=False, adapt_movement_scale=True) for p in imgs]
HTML(display2(driving, imgs, preds).to_html5_video())

100%|████████████████████████████████████████████████████████████████████████████████| 211/211 [00:12<00:00, 16.47it/s]
